In [31]:
# -*- coding: utf-8 -*-

"""
This module parses and analyzes the data from Area Health Resource Files (ARHF)
This notebook has been auto-formatted for PEP8 coding standards using the autopep8 NBExtensions for Jupyter Notebook

@author: Aishwarya Ramakrishnan

"""

# Import necessary modules
import os
import re
import itertools
import pandas as pd
import numpy as np
import matplotlib as mpl
from openpyxl import load_workbook
import logging
import datetime as dt

# Set Log levels for debugging
logger = logging.getLogger()
logging.basicConfig(
    format="%(asctime)s [%(levelname)s|%(lineno)d]\n%(message)s", level=logging.DEBUG)  # ERROR,WARNING,INFO,DEBUG

In [32]:
class ARHFTechDoc:
    """
    A class to read and parse the ARHF Technical Documentation that contains information about the header
    and format of the ARHF data
    """

    def __init__(self):
        """
        Open and read the xlsx file into a df
        Inputs: None
        Returns: None
        """
        header_names = ["field", "col-col", "year_of_data",
                        "variable_name", "characteristics"]
        self.techdoc_df = pd.read_excel(
            'AHRF 2018-2019 Technical Documentation.xlsx', skiprows=183, header=None, names=header_names, usecols="A:E",
            dtype=object,
            keep_default_na=False)
        logging.debug(self.techdoc_df.shape)
        self.__clean_techdoc_data()

    def __clean_techdoc_data(self):
        """
        Clean the ARHF technical doc dataframe by removing unwanted excel rows
        Inputs: None
        Returns: None
        """
        self.techdoc_df['field'] = self.techdoc_df['field'].apply(lambda x: x.strip())   #Strip all whitespaces
        self.techdoc_df['variable_name'] = self.techdoc_df['variable_name'].apply(lambda x: x.strip())   
        
        self.techdoc_df = self.techdoc_df[self.techdoc_df["field"].str.contains(
            "F[0-9]{5}.*", na=False)]  # Filter rows with field starting with 'F'

        #Convert dates like '2006-10' to list ['2006','2007','2008','2009','2010']
        def generate_year_range(date_str):
            start_yr = date_str[0:4]
            end_yr = date_str[0:2]+date_str[5:]
            year_range = pd.period_range(start_yr,end_yr, freq="1Y")   #Generate period range from start year to end year
            year_list = year_range.strftime('%Y').tolist()   #Extract year values as a list
            year_list = [int(i) for i in year_list]   #Convert to int
            return year_list

        logging.debug(generate_year_range('2006-10'))    #Test code
        for index, rows in self.techdoc_df.iterrows():
            if "-" in str(self.techdoc_df.loc[index, "year_of_data"]):
                self.techdoc_df.loc[index, "year_of_data"] = generate_year_range(str(self.techdoc_df.loc[index, "year_of_data"]))
        self.techdoc_df = self.techdoc_df.explode('year_of_data', ignore_index=True)  #Expand the list of years into separate rows


    def get_variable_names(self,is_unique=False, include_date=False):
        """
        Returns the applicable field names of the data
        Inputs: is_unique - if unique variable names are expected to be returned
        Returns: Series of all 'variable_name'
        """
        if is_unique:
            return self.techdoc_df['variable_name'].unique().str.strip()
        elif include_date:
            res_df = self.techdoc_df['variable_name'] + "::"+  self.techdoc_df['year_of_data'].astype(str)
            return res_df.tolist()
        else:
            return self.techdoc_df['variable_name'].str.strip()

    def get_byte_ranges(self):
        return self.techdoc_df['col-col'].tolist()
    
    def get_indices_of_cols(self, col_names):
        indices=[]
        if not isinstance(col_names, list):
            col_names = [col_names]
        for item in col_names:
            indices.extend(self.techdoc_df.index[self.techdoc_df['variable_name'].str.contains(item)].to_list())
        return indices

In [33]:
class ARHFAsciiData:
    import re
    """
       A class to read and parse the ARHF ASCII file that contains the data which needs to be parsed correctly to the
        headers obtained from ARHFTechnical Document
       """

    def __init__(self, column_headers, byte_range_for_col_headers):
        #Check if number of columns and number of byte ranges provided are equal
        if not isinstance(column_headers, list):
            column_headers = [column_headers]
        if not isinstance(byte_range_for_col_headers, list):
            byte_range_for_col_headers = [byte_range_for_col_headers]
        assert len(column_headers)==len(byte_range_for_col_headers), "Number of columns is not equal to number of byte values provided!"
        
        self.columns = [name.strip() for name in column_headers]
        self.byte_range_for_cols = byte_range_for_col_headers

    def __extract_bytes(self, line, bytes):
        if not isinstance(bytes, str):
            bytes = bytes.decode()
        range = re.split('-', bytes)
        start_byte = int(range[0])-1
        end_byte = int(range[1])
        return line[start_byte:end_byte]

    def read_and_parse_data_to_csv(self, data_file="AHRF2019.asc", output_file="AHRF2019_processed.csv"):
        # Prepare the output csv file by writing the column headers first
        data_df = pd.DataFrame(columns=self.columns)
        self.append_df_to_csv(data_df, output_file, include_header=True)

        # Open data file and parse line by line
        with open(data_file) as fp:
            rows = []
            for cnt, line in enumerate(fp):
                row_values = []
                if isinstance(self.byte_range_for_cols, list):
                    for item in self.byte_range_for_cols:
                        row_values.append(self.__extract_bytes(line, item))
                else:
                    row_values.append(self.__extract_bytes(
                        line, self.byte_range_for_cols))
                rows.append(pd.Series(row_values))
                if cnt % 100 == 0:
                    logging.debug("Reading line %d", cnt)
                    df = pd.DataFrame(rows)
                    # Write parsed data to csv without headers
                    self.append_df_to_csv(df, output_file)
                    rows.clear()
        
        if len(rows)>0:    # Ensure to save last few rows
            logging.debug("Reading line %d", cnt)
            df = pd.DataFrame(rows)
            self.append_df_to_csv(df, output_file)
            rows.clear()
            
    def append_df_to_csv(self, df, file_name, include_header=False):
        df.to_csv(file_name, mode='a', header=include_header, index=False)

In [35]:
techdoc_obj = ARHFTechDoc()

# Test code to check Assertion error of incompatible column and byte lengths
#column_list = techdoc_obj.get_variable_names(include_date=True)
#column_byte_list = ["00001-00001","00002-00005"]
#data_obj = ARHFAsciiData(column_list, column_byte_list)

column_list = techdoc_obj.get_variable_names(include_date=True)
column_byte_list = techdoc_obj.get_byte_ranges()
data_obj = ARHFAsciiData(column_list, column_byte_list)
data_obj.read_and_parse_data_to_csv()

2020-11-01 23:43:43,629 [DEBUG|19]
(8563, 5)
2020-11-01 23:43:43,644 [DEBUG|43]
[2006, 2007, 2008, 2009, 2010]
2020-11-01 23:43:44,966 [DEBUG|45]
Reading line 0
2020-11-01 23:43:47,093 [DEBUG|45]
Reading line 100
2020-11-01 23:43:49,695 [DEBUG|45]
Reading line 200
2020-11-01 23:43:52,434 [DEBUG|45]
Reading line 300
2020-11-01 23:43:55,271 [DEBUG|45]
Reading line 400
2020-11-01 23:43:58,080 [DEBUG|45]
Reading line 500
2020-11-01 23:44:00,989 [DEBUG|45]
Reading line 600
2020-11-01 23:44:04,044 [DEBUG|45]
Reading line 700
2020-11-01 23:44:06,970 [DEBUG|45]
Reading line 800
2020-11-01 23:44:10,175 [DEBUG|45]
Reading line 900
2020-11-01 23:44:13,203 [DEBUG|45]
Reading line 1000
2020-11-01 23:44:16,242 [DEBUG|45]
Reading line 1100
2020-11-01 23:44:19,403 [DEBUG|45]
Reading line 1200
2020-11-01 23:44:22,429 [DEBUG|45]
Reading line 1300
2020-11-01 23:44:26,303 [DEBUG|45]
Reading line 1400
2020-11-01 23:44:29,779 [DEBUG|45]
Reading line 1500
2020-11-01 23:44:32,776 [DEBUG|45]
Reading line 1600
